In [ ]:
from tbh import runner_tools as rt 
from tbh import plotting as pl

from importlib import reload
reload(rt);
reload(pl);

In [ ]:
studies_dict = rt.DEFAULT_STUDIES_DICT
params = rt.get_full_default_params(studies_dict)

# Manual run

In [ ]:
model, do_df = rt.model_single_run(rt.DEFAULT_MODEL_CONFIG, studies_dict, params)

In [ ]:
do_df[['ltbi_propXmajuro']].plot()

# Calibration and full runs

In [ ]:
# Metropolis config
tune = 50
draws = 100

# Full runs config
burn_in = int(draws / 2.) # 10000
full_runs_samples = 50

In [ ]:
bcm = rt.get_bcm_object(rt.DEFAULT_MODEL_CONFIG, studies_dict, params)

In [ ]:
idata = rt.run_metropolis_calibration(bcm, draws=draws, tune=tune)

In [ ]:
import arviz as az

rhats = az.rhat(idata)
burnt_rhats = az.rhat(idata.sel(draw=range(burn_in, idata.sample_stats.sizes['draw'])))
print(f"Max R_hat for full chains: {rhats.to_array().max().item()}")
print(f"Max R_hat for burnt chains: {burnt_rhats.to_array().max().item()}")

In [ ]:
full_runs, unc_df = rt.run_full_runs(bcm, idata, burn_in, full_runs_samples)

# Visualise traces and posteriors

In [ ]:
pl.plot_traces(idata, burn_in)

In [ ]:
pl.plot_post_prior_comparison(idata, burn_in, list(bcm.priors.keys()), list(bcm.priors.values()), n_col=4)

# Plot outputs with uncertainty

In [ ]:
import matplotlib.pyplot as plt

selected_outputs = bcm.targets.keys()

for output in selected_outputs:
    fig, ax = plt.subplots()
    pl.plot_model_fit_with_uncertainty(ax, unc_df, output, bcm, x_min=2010)